In [4]:
#! pip install jsonrpcclient

from jsonrpcclient import request, parse, Ok
import logging
import requests
import pandas as pd
import numpy as np


In [72]:
url = "https://muddy-falling-river.solana-mainnet.quiknode.pro/f6caf7f878f08e5b5b1d88aa3eff9238693e525a/"

def pull_traits(creator_address,url=url, start_page=1): 

    pg_num = start_page
    search = True #sam thinks this a good idea ....

    all_results = []

    while search:
        response = requests.post(
            url=url,
            json=request(
                "qn_fetchNFTsByCreator",
                [
                    {
                        "creator": creator_address,
                        "page": pg_num,
                        "perPage": 10000,
                    }
                ],
            ),
        )
        parsed = parse(response.json())
        print('page ='+str(pg_num))
        pg_num+=1
        
        #print(len(parsed.result["assets"]))

        if pg_num > int(parsed.result['totalPages']):
            search = False
        if isinstance(parsed, Ok) and len(parsed.result["assets"])>=1:
            #print("yay")
            all_results.append(parsed.result["assets"])
        else:
            print('something gone wrong'
            )
            print(parsed)
            logging.error(parsed.message)
            search = False
        

    return all_results


In [73]:
def get_collection_traits(collection_address, collection_name):
    #pull data from api
    json_list = pull_traits(creator_address=collection_address)
    #pull traits returns a nested list jsons
    #convert to 1 dimensional list
    result = []
    for sublist in json_list:
        for item in sublist:
            result.append(item)
    
    #dataframe with all data
    print('stage 1')
    df_all = pd.DataFrame.from_records(result)
    #dataframe with traits only
    print('stage 2')
    df_traits = pd.json_normalize(result, 'traits', 
                        record_prefix='traits_',meta='name').drop_duplicates()

    #get unique list of trait types
    print('stage 3')
    trait_list = df_traits.traits_trait_type.unique()
    
    #pivot it by asset
    print('stage 4')
    print(df_traits)
    assets_traits = df_traits.pivot(index='name', columns="traits_trait_type", values="traits_value")

    #calculate rarity of each trait
    print('stage 5')
    for t in trait_list:
        i_proportion = assets_traits[t].value_counts(dropna=False, normalize=True)
        
        assets_traits = assets_traits.merge(
                i_proportion,
                left_on=t,
                right_index=True,
                suffixes=("", "_rarity"),
                how="left",
            )

    
    #calculate overall rarity
    print('stage 6')
    rarity_cols = [s + '_rarity' for s in trait_list]

    print('stage 7')
    rarity = assets_traits.copy()
    #calculate minumim rarity (most rare trait)
    rarity['min_rarity']= rarity[rarity_cols].min(axis=1)
    #calculate maximum rarity (most rare trait)
    rarity['max_rarity']= rarity[rarity_cols].max(axis=1)
    rarity['mean_rarity']= rarity[rarity_cols].mean(axis=1)
    #calculate factoral of each individual trait rarity score
    factorial_rarity = 1 / rarity[rarity_cols]
    #sum the factoral rarities to give overall rarity score
    rarity["factoral_rarity"] = factorial_rarity.sum(axis=1)
    #score values by overall (factoral rarity)
    rarity = rarity.sort_values("factoral_rarity", ascending=False)
    #give rank
    rarity["rarity_rank"] = np.arange(1, rarity.shape[0] + 1, 1)
    names = rarity.index
    rarity=rarity.reset_index()
    #pull out from name
    #rarity['name']=rarity.index
    rarity['id']= rarity.name.str.extract('(\d+)').astype('int')

    other_colums = ['tokenAddress','collectionAddress','collectionName','imageUrl','chain','name']
    rarity= rarity.merge(df_all[other_colums],how='outer', on='name')

    rarity.to_csv(f"{collection_name}_traits.csv")

    return rarity

    


In [57]:
de_gods = '9MynErYQ5Qi6obp4YwwdoDmXkZ1hYVtPUqYmJJ3rZ9Kn'

de_gods_rarity = get_collection_traits(collection_address=de_gods, collection_name='DeGods')

page =1
page =2
page =3
page =4
page =5
page =6
page =7
page =8
page =9
page =10
page =11
page =12
page =13
page =14
page =15
page =16
page =17
page =18
page =19
page =20
page =21
page =22
page =23
page =24
page =25
page =26
page =27
page =28
page =29
page =30
page =31
page =32
page =33
page =34
page =35
page =36
page =37
page =38
page =39
page =40
page =41
page =42
page =43
page =44
page =45
page =46
page =47
page =48
page =49
page =50
page =51
page =52
page =53
page =54
page =55
page =56
page =57
page =58
page =59
page =60
page =61
page =62
page =63
page =64
page =65
page =66
page =67
page =68
page =69
page =70
page =71
page =72
page =73
page =74
page =75
page =76
page =77
page =78
page =79
page =80
page =81
page =82
page =83
page =84
page =85
page =86
page =87
page =88
page =89
page =90
page =91
page =92
page =93
page =94
page =95
page =96
page =97
page =98
page =99
page =100
page =101
page =102
page =103
page =104
page =105
page =106
page =107
page =108
page =109
page =110
page =11

In [68]:
rarity.shape

(7501, 32)

In [70]:
okay_bears = "3xVDoLaecZwXXtN59o6T3Gfxwjcgf8Hc9RfoqBn995P9"
bears_rarity = get_collection_traits(collection_address= okay_bears, collection_name='OkayBears')

page =1
page =2
page =3
page =4
page =5
page =6
page =7
page =8
page =9


KeyboardInterrupt: 